In [10]:
#Importing everything we need for EDA. Black for linting reasons.
import os
import black
import jupyter_black
from collections import Counter
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import xml.etree.ElementTree as ET
import pandas as pd
import nltk
import json
import fasttext
pd.options.mode.chained_assignment = None  # default='warn'
nltk.download("stopwords")
nltk.download("punkt")
jupyter_black.load(
    lab=False,
    line_length=80,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cacu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cacu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
csv_file_path = r"C:\Users\Cacu\Desktop\Universidad\Trabajo_Final\DataSets\open-dataset-for-sentiment-analysis-master\betsentiment-ES-tweets-sentiment-worldcup.csv"


# Function to extract sentiment scores from the json str
def extract_sentiment_scores(json_str):
    sentiment_data = json.loads(json_str)
    return (
        sentiment_data["Neutral"],
        sentiment_data["Negative"],
        sentiment_data["Positive"],
        sentiment_data["Mixed"],
    )


# Define data types
dtype_dict = {
    "tweet_date_created": str,
    "tweet_id": int,
    "tweet_text": str,
    "language": str,
    "sentiment": str,
}

try:
    df = pd.read_csv(csv_file_path, encoding="utf-8", dtype=dtype_dict)
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_path, encoding="latin-1", dtype=dtype_dict)

sentiment_scores_list = df["sentiment_score"].map(extract_sentiment_scores)

sentiment_scores_df = pd.DataFrame(
    sentiment_scores_list.tolist(),
    columns=["Neutral", "Negative", "Positive", "Mixed"],
)

df = pd.concat([df, sentiment_scores_df], axis=1)

df.drop("sentiment_score", axis=1, inplace=True)

In [3]:
df

,tweet_date_created,tweet_id,tweet_text,language,sentiment,Neutral,Negative,Positive,Mixed
0,2018-06-17T09:47:48,-136978431,@JoseAMeadeK @miseleccionmx A pero si estás en todo menos en tu campaña y todavía te preguntas porque vas en tercer lugar,es,NEUTRAL,0.597391,0.205260,0.085440,0.111909
1,2018-06-28T06:00:16.360000,-1076408320,"#MarioPereyraDT\n""Tenemos que jugarle a Francia con el mismo equipo"" https://t.co/15ccsyMtOi \n\n#SomosArgentina #Cadena3Mundial https://t.co/aNUZMjDkaR",es,NEUTRAL,0.878757,0.058990,0.049931,0.012322
2,2018-06-07T22:07:43,1008168960,"@miseleccionmx No me pidas eso mi selección, sí a eso voy a los estadios a sacar el FUA.",es,NEGATIVE,0.315568,0.556793,0.056737,0.070902
3,2018-05-31T21:02:10,-1550348286,"Si llega a ser la despedida, no será la mejor...\nPero casi que quiero obligar a todos a mirar 4 años atrás y ver todo lo que, atrás de este tipo, lograron jugadores, dirigentes y todos los que forman parte de @Argentina \nGracias por hacernos mejores (y felices) personas????? https://t.co/amTgw75rdq",es,POSITIVE,0.244512,0.005207,0.686508,0.063773
4,2018-06-26T11:02:06,-1604923392,No se les olvide que nuestro trabajo es construir la dictadura del proletariado.\n#VamosColombia,es,NEUTRAL,0.424387,0.418878,0.085919,0.070817
...,...,...,...,...,...,...,...,...,...
1072456,2018-06-17T14:12:56,-1932234751,Se llegó el dia @miseleccionmx pongan el nombre de México en alto! \nTodos estamos con ustedes.???\n#NadaNosDetiene,es,NEUTRAL,0.394735,0.376690,0.160512,0.068063
1072457,2018-06-17T14:12:56.857000,-1932234751,Se llegó el dia @miseleccionmx pongan el nombre de México en alto! \nTodos estamos con ustedes.???\n#NadaNosDetiene,es,NEUTRAL,0.394735,0.376690,0.160512,0.068063
1072458,2018-05-14T01:54:14,-677941248,"@miseleccionmx @ClubAmerica Oribe Peralta no merece ser de la selección nacional, jugador no es, es ratero por solo cobrar nomina sin rendimiento alguno.",es,NEGATIVE,0.286460,0.603919,0.054673,0.054948
1072459,2018-06-23T15:22:55,-1139363840,mi hermano cree q por hacerme sonidos raros cada que sale un coreano me voy a molestar\n#ElMariachiHaráBailarACorea #COREAndoAlaFinal\n#MexicoVsCorea \n#NadaNosDetiene\n#VamosMéxico,es,NEGATIVE,0.166471,0.757941,0.028321,0.047267


In [4]:
# Read the stopwords from the .txt file and store them in a list
custom_stopwords_file = (
    r"C:\Users\Cacu\Desktop\Universidad\Trabajo_Final\utilities\stopwords.txt"
)
with open(custom_stopwords_file, "r", encoding="utf-8") as file:
    custom_stopwords_list = [line.strip() for line in file]

# Create an empty set to hold the stopwords
custom_stopwords_set = set()

# Add the stopwords from the list to the set
custom_stopwords_set.update(custom_stopwords_list)

In [7]:
# Specify the Snowball stemmer for Spanish
stemmer = SnowballStemmer("spanish")

# Convert NaN values to an empty string
df["tweet_text"] = df["tweet_text"].fillna("")

# Lowercasing
df["tweet_text"] = df["tweet_text"].str.lower()

# Removing punctuation
df["tweet_text"] = df["tweet_text"].str.replace("[^\w\s]", "")

# Tokenization
df["tokens"] = df["tweet_text"].apply(word_tokenize)

# Removing stop words
stop_words = set(stopwords.words("spanish"))
df["filtered_tokens"] = df["tokens"].apply(
    lambda tokens: [word for word in tokens if word not in stop_words]
)

# Stemming
df["stemmed_tokens"] = df["filtered_tokens"].apply(
    lambda tokens: [stemmer.stem(word) for word in tokens]
)

C:\Users\Cacu\AppData\Local\Temp\ipykernel_9560\709154844.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df["tweet_text"] = df["tweet_text"].str.replace("[^\w\s]", "")


In [8]:
df

,tweet_date_created,tweet_id,tweet_text,language,sentiment,Neutral,Negative,Positive,Mixed,tokens,filtered_tokens,stemmed_tokens
0,2018-06-17T09:47:48,-136978431,joseameadek miseleccionmx a pero si estás en todo menos en tu campaña y todavía te preguntas porque vas en tercer lugar,es,NEUTRAL,0.597391,0.205260,0.085440,0.111909,"[joseameadek, miseleccionmx, a, pero, si, estás, en, todo, menos, en, tu, campaña, y, todavía, te, preguntas, porque, vas, en, tercer, lugar]","[joseameadek, miseleccionmx, si, menos, campaña, todavía, preguntas, vas, tercer, lugar]","[joseameadek, miseleccionmx, si, men, campañ, todav, pregunt, vas, terc, lug]"
1,2018-06-28T06:00:16.360000,-1076408320,mariopereyradt\ntenemos que jugarle a francia con el mismo equipo httpstco15ccsymtoi \n\nsomosargentina cadena3mundial httpstcoanuzmjdkar,es,NEUTRAL,0.878757,0.058990,0.049931,0.012322,"[mariopereyradt, tenemos, que, jugarle, a, francia, con, el, mismo, equipo, httpstco15ccsymtoi, somosargentina, cadena3mundial, httpstcoanuzmjdkar]","[mariopereyradt, jugarle, francia, mismo, equipo, httpstco15ccsymtoi, somosargentina, cadena3mundial, httpstcoanuzmjdkar]","[mariopereyradt, jug, franci, mism, equip, httpstco15ccsymtoi, somosargentin, cadena3mundial, httpstcoanuzmjdk]"
2,2018-06-07T22:07:43,1008168960,miseleccionmx no me pidas eso mi selección sí a eso voy a los estadios a sacar el fua,es,NEGATIVE,0.315568,0.556793,0.056737,0.070902,"[miseleccionmx, no, me, pidas, eso, mi, selección, sí, a, eso, voy, a, los, estadios, a, sacar, el, fua]","[miseleccionmx, pidas, selección, voy, estadios, sacar, fua]","[miseleccionmx, pid, seleccion, voy, estadi, sac, fua]"
3,2018-05-31T21:02:10,-1550348286,si llega a ser la despedida no será la mejor\npero casi que quiero obligar a todos a mirar 4 años atrás y ver todo lo que atrás de este tipo lograron jugadores dirigentes y todos los que forman parte de argentina \ngracias por hacernos mejores y felices personas httpstcoamtgw75rdq,es,POSITIVE,0.244512,0.005207,0.686508,0.063773,"[si, llega, a, ser, la, despedida, no, será, la, mejor, pero, casi, que, quiero, obligar, a, todos, a, mirar, 4, años, atrás, y, ver, todo, lo, que, atrás, de, este, tipo, lograron, jugadores, dirigentes, y, todos, los, que, forman, parte, de, argentina, gracias, por, hacernos, mejores, y, felices, personas, httpstcoamtgw75rdq]","[si, llega, ser, despedida, mejor, casi, quiero, obligar, mirar, 4, años, atrás, ver, atrás, tipo, lograron, jugadores, dirigentes, forman, parte, argentina, gracias, hacernos, mejores, felices, personas, httpstcoamtgw75rdq]","[si, lleg, ser, desped, mejor, casi, quier, oblig, mir, 4, años, atras, ver, atras, tip, logr, jugador, dirigent, form, part, argentin, graci, hac, mejor, felic, person, httpstcoamtgw75rdq]"
4,2018-06-26T11:02:06,-1604923392,no se les olvide que nuestro trabajo es construir la dictadura del proletariado\nvamoscolombia,es,NEUTRAL,0.424387,0.418878,0.085919,0.070817,"[no, se, les, olvide, que, nuestro, trabajo, es, construir, la, dictadura, del, proletariado, vamoscolombia]","[olvide, trabajo, construir, dictadura, proletariado, vamoscolombia]","[olvid, trabaj, constru, dictadur, proletari, vamoscolombi]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1072456,2018-06-17T14:12:56,-1932234751,se llegó el dia miseleccionmx pongan el nombre de méxico en alto \ntodos estamos con ustedes\nnadanosdetiene,es,NEUTRAL,0.394735,0.376690,0.160512,0.068063,"[se, llegó, el, dia, miseleccionmx, pongan, el, nombre, de, méxico, en, alto, todos, estamos, con, ustedes, nadanosdetiene]","[llegó, dia, miseleccionmx, pongan, nombre, méxico, alto, ustedes, nadanosdetiene]","[lleg, dia, miseleccionmx, pong, nombr, mexic, alto, usted, nadanosdetien]"
1072457,2018-06-17T14:12:56.857000,-1932234751,se llegó el dia miseleccionmx pongan el nombre de méxico en alto \ntodos estamos con ustedes\nnadanosdetiene,es,NEUTRAL,0.394735,0.376690,0.160512,0.068063,"[se, llegó, el, dia, miseleccionmx, pongan, el, nombre, de, méxico, en, a

## 1. Embedding

In [13]:
df['tokens_as_string'] = df['tokens'].apply(' '.join)
# Save the stemmed tokens to a text file (one sentence per line)
with open('tokens.txt', 'w') as f:
    f.write('\n'.join(df['tokens_as_string']))

AttributeError: 'list' object has no attribute 'find'

In [ ]:
# Train a FastText model on the stemmed tokens
model = fasttext.train_unsupervised('tokens.txt', model='skipgram')

# Create tweet-level embeddings using the trained model
embeddings = []

In [14]:
for tokens in df['tokens_as_string']:
    vector = model.get_sentence_vector(tokens)
    embeddings.append(vector)

# Convert the embeddings to a DataFrame
embedding_df = pd.DataFrame(embeddings)


NameError: name 'df_worldcup_embeddings' is not defined

In [ ]:
# Concatenate the embeddings DataFrame with the original dataset
df_worldcup_embeddings = pd.concat([df, embedding_df], axis=1)

# Save the dataset with embeddings to a new CSV file
df_worldcup_embeddings.to_csv('2018_dataset_with_embeddings.csv', index=False)